In [1]:
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
import pandas as pd
import shoe
import blackjack

In [2]:
#DEPRICATED
#cards DICTIONARY IS NOW STORED IN blackjack.py
#Dictionary of Blackjack Cards
cards = {'two':2,
        'three':3,
        'four':4,
        'five':5,
        'six':6,
        'seven':7,
        'eight':8,
        'nine':9,
        'ten':10,
        'jack':10,
        'queen':10,
        'king':10,
        'ace':[11,1]}

In [8]:
#DEPRICATED
#CreateDeck METHOD IS NOW IN blackjack.py
def CreateDeck():
    #RETURNS: deck - list of str of cards making a standard 52 card deck
    deck = list(cards.keys())*4
#     random.shuffle(deck) #inplace shuffle
    
    return deck

In [21]:
print(f'2 deck cut boundaries: {10/(52*2)} to {15/(52*2)}')
print(f'6 deck cut boundaries: {52/(52*6)} to {78/(52*6)}')
print(f'Dealer always penetrates at a range of 1 to 1.5 decks from the back, or {52/(52*6)} to {78/(52*6)}')

2 deck cut boundaries: 0.09615384615384616 to 0.14423076923076922
6 deck cut boundaries: 0.16666666666666666 to 0.25
Dealer always penetrates at a range of 1 to 1.5 decks from the back, or 0.16666666666666666 to 0.25


In [25]:
#DEPRICATED
#UPDATED VERSION IN shoe.py

def CreateShoe(deck,shoe_size = 6):
    '''
    PARAMETERS:
    shoe_size - int of number of decks in shoe

    RETURNS:
    tuple - list of [shoe_size * deck] cards minus random penetration
            and
            list of cards equal to penetration (reserve cards for dealing last hand)
    '''

    if shoe_size > 2:
        pen_range = [(52/(52*shoe_size)), ((52*1.5)/(52*shoe_size))]
    else:
        #single and double deck penetrate from 10 to 15 cards
        pen_range = [10/(52*shoe_size), 15/(52*shoe_size)]

    #ensures that each deck added to the shoe is shuffled
    #this might just be gamblers superstition on my part...
    shoe = []
    for _ in range(shoe_size):
        random.shuffle(deck)
        shoe += deck
        
    random.shuffle(shoe) #in-place shuffle
    
    #generates penetration at a random specific card based on pen_range
    penetration = round(len(shoe)*random.uniform(pen_range[0],pen_range[1]))
    #generates playbale shoe with cards equal to penetration taken out
    
    return shoe[:-penetration],shoe[-penetration:]

In [4]:
deck = blackjack.CreateDeck()
# type(deck)

In [2]:
playable_shoe = shoe.Shoe()
playable_shoe.CreateShoe(deck)

NameError: name 'deck' is not defined

In [31]:
#Debug purposes only
print(len(playable_shoe._shoe))
card = playable_shoe.DrawCard()
print(card)
print(len(playable_shoe._shoe))

239
eight
238


In [76]:
#DEPRICATED
#GetHandValue IS NOW A METHOD IN blackjack.py
def GetHandValue(hand):
    '''
    Gets the value of a single blackjack hand
    
    PARAMETERS:
    hand - list of cards as str corresponding with cards dictionary
    
    RETURNS:
    hand_value - int in hand value (highest possible hand value if aces are present)
    '''
    
    
    hand_value = 0
    num_aces = hand.count('ace') #gets number of aces
    aces_value = num_aces #assumes all aces are 1's for initial calculation
    
    hand.sort(reverse=True) #sorts all aces to back of array
    for card in hand[:len(hand)-num_aces]: #iterates through list without aces
        hand_value += cards[card] #calculates hand value pre-aces
    
    for i in range(num_aces+1): #iterates +1 in case there is only 1 ace
        if hand_value + ((i*11) + (num_aces-i)) <= 21: #checks combinations of 11s and 1s
            aces_value = (i*11) + (num_aces-i) #sets combination of 11s and 1s if not a bust
        else: #breaks out of loop once the maximum 11s:1s ratio is found
            break
        
    hand_value += aces_value
    
    if hand_value > 21:
        return -1 #-1 is read as a bust when returned
    else:
        return hand_value

In [2]:
def PlayBlackjack():
    '''
    Plays blackjack
    
    No parameters or returns. This is a self-contained method
    that plays blackjack with while loops for user input.
    '''
    
    #verbose statement
    print('Creating shoe...')
    
    #####################################################################
    deck = blackjack.CreateDeck() #Remove blackjack. once copied into blackjack.py
    #####################################################################
    shoe_obj = shoe.Shoe()
    
    
    playing = True
    while playing:
        #checks to see if shoe in the shoe class is empty
        #if empty, it will create a new shoe
        if len(shoe_obj._shoe) == 0:
            #verbose statement
            print('Filling new shoe with cards...')
            shoe_obj.CreateShoe(deck)
        
        in_hand = True
        while in_hand:
            #verbose statement
            print('New hand start!')
            
            p_hand = {'cards':[],
                      'value':0}
            d_hand = {'cards':[],
                      'shown_value':0,
                      'true_value':0}
            
            print('Initial draw...')
            
            #draws first 4 cards
            #TODO
            #create a list of player dictionaries and iterate through
            #rather than hard-code the number of draws to make
            for _ in range(2):
                ######
                #TODO: Implement empty shoe check
                ######
                p_hand['cards'].append(shoe_obj.DrawCard())
                
                d_hand['cards'].append(shoe_obj.DrawCard())
                
            p_hand['value'] = blackjack.GetHandValue(p_hand['cards'])
            
            #need to give GetHandValue a list, hence the brackets
            d_hand['shown_value'] = blackjack.GetHandValue([d_hand['cards'][0]])
            d_hand['true_value'] = blackjack.GetHandValue(d_hand['cards'])
            
            print(f"Players hand: {p_hand['cards']}\nPlayers hand value: {p_hand['value']}")
            print(f"Dealers hand: {[d_hand['cards'][0]]}\nDealers hand value: {d_hand['shown_value']}")    
            
            '''
            Below is a huge block of if statements that are checking
            for player and dealer blackjack cases.
            '''
            if p_hand['value'] == 21:
                print('Blackjack!')
                
                if d_hand['shown_value'] == 11:
                    even_money = input('Take even money? (y/n) ')
                    if even_money == 'y':
                        print('Paid player 1x bet.')
                        #TODO Pay player 1x
                    else:
                        print("We're playing!")
                        
                else:
                    print('Paid player 1.5x bet.')
                    #TODO Pay player 1.5x for Blackjack
                    player_acting = False
                    
            if d_hand['shown_value'] == 11:
                insurance = input('Insurance? (y/n) ')
                if d_hand['true_value'] == 21:
                    print(f'Dealer has blackjack with {d_hand["cards"]}')
                    if insurance == 'y':
                        print("Player has insurance.")
                        #TODO Pay player insurance
                    else:
                        print("Player does not have insurance.")
                        #TODO Take player bet
                    player_acting = False
                else:
                    print('Dealer does not have blackjack!')
                    
            else:
                player_acting = True
            '''
            End of blackjack-checking if statements.
            '''
            
#             while player_acting:
                
#                 input('Test')
                
                

In [3]:
PlayBlackjack()

Creating shoe...
Filling new shoe with cards...
Shuffling 6 decks into shoe...
Cutting...
Burning first card...
Drawing card from shoe...
Shoe is read to play!
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Players hand: ['ace', 'ten']
Players hand value: 21
Dealers hand: ['five']
Dealers hand value: 5
Blackjack!
Paid player 1.5x bet.
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Players hand: ['queen', 'ten']
Players hand value: 20
Dealers hand: ['nine']
Dealers hand value: 9
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Players hand: ['nine', 'nine']
Players hand value: 18
Dealers hand: ['eight']
Dealers hand value: 8
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from s

Insurance? (y/n)  y


Dealer does not have blackjack!
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Players hand: ['ace', 'three']
Players hand value: 14
Dealers hand: ['four']
Dealers hand value: 10
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Players hand: ['eight', 'three']
Players hand value: 11
Dealers hand: ['nine']
Dealers hand value: 9
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Players hand: ['five', 'six']
Players hand value: 11
Dealers hand: ['jack']
Dealers hand value: 10
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Players hand: ['five', 'seven']
Players hand value: 12
Dealers hand: ['five']
Dealers hand value: 5
New hand start!
Initial 

Insurance? (y/n)  n


Dealer does not have blackjack!
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Players hand: ['six', 'three']
Players hand value: 9
Dealers hand: ['jack']
Dealers hand value: 10
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Players hand: ['ten', 'three']
Players hand value: 13
Dealers hand: ['ace']
Dealers hand value: 9
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Players hand: ['ace', 'eight']
Players hand value: 19
Dealers hand: ['jack']
Dealers hand value: 10
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Players hand: ['four', 'queen']
Players hand value: 14
Dealers hand: ['six']
Dealers hand value: 6
New hand start!
Initial draw

...

Insurance? (y/n)  n


Dealer has blackjack with ['ace', 'jack']
Player does not have insurance.
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Players hand: ['nine', 'queen']
Players hand value: 19
Dealers hand: ['five']
Dealers hand value: 6
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Players hand: ['queen', 'two']
Players hand value: 12
Dealers hand: ['five']
Dealers hand value: 7
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Players hand: ['eight', 'five']
Players hand value: 13
Dealers hand: ['eight']
Dealers hand value: 8
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Players hand: ['king', 'two']
Players hand value: 12
Dealers hand: ['ace']
Dealers

Insurance? (y/n)  n


Dealer does not have blackjack!
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Players hand: ['ace', 'nine']
Players hand value: 20
Dealers hand: ['five']
Dealers hand value: 5
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Players hand: ['ace', 'seven']
Players hand value: 18
Dealers hand: ['four']
Dealers hand value: 4
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Players hand: ['king', 'queen']
Players hand value: 20
Dealers hand: ['five']
Dealers hand value: 5
New hand start!
Initial draw...
Drawing card from shoe...
Drawing card from shoe...
Drawing card from shoe...
Shoe is now empty. Drawing from cut reserve...
Players hand: ['four', 'ten']
Players hand value: 14
Dealers hand: ['five']
Dealers hand value: 5
New hand

IndexError: pop from empty list